## 1. Setup và Import Libraries

In [1]:
# Import essential libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set random seed cho reproducible results
import random
import os
random.seed(42)
np.random.seed(42)
os.environ['PYTHONHASHSEED'] = '42'

print("✅ Libraries imported successfully!")
print(f"📅 Current date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ Libraries imported successfully!
📅 Current date: 2025-12-03 02:19:05


In [2]:
# Import project modules
import sys
sys.path.append('src')

try:
    from data_loader import get_prices_for_list, compute_returns, get_market_index_returns
    from analysis_basic import summarize_all_stocks
    from efficient_frontier import EfficientFrontierAnalyzer
    from capm import CAPMAnalyzer
    from lstm_forecast import run_lstm_analysis, simple_moving_average_forecast
    
    print("✅ All project modules imported successfully!")
    
except ImportError as e:
    print(f"⚠️ Module import error: {e}")
    print("🔧 Make sure all source files are in the 'src' directory")

2025-12-03 02:19:14.543 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-12-03 02:19:14.559 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-12-03 02:19:14.559 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-12-03 02:19:14.559 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


✅ All project modules imported successfully!


## 2. Cấu hình Phân tích

In [3]:
# Configuration parameters
CONFIG = {
    'symbols': ['FPT', 'VNM', 'HPG', 'MWG', 'VCB'],
    'start_date': '2022-01-01',
    'end_date': '2025-12-03', 
    'rf_rate': 0.03,  # Risk-free rate 3%
    'lookback_years': 1,  # LSTM lookback period (1-3 years)
    'forecast_days': 30,  # LSTM forecast period
    'investment_amount': 10_000_000  # 10M VND
}

print("📊 Analysis Configuration:")
print("=" * 40)
for key, value in CONFIG.items():
    print(f"{key:20}: {value}")

# Extract variables for convenience
SYMBOLS = CONFIG['symbols']
START_DATE = CONFIG['start_date']
END_DATE = CONFIG['end_date']
RF_RATE = CONFIG['rf_rate']
LOOKBACK_YEARS = CONFIG['lookback_years']
FORECAST_DAYS = CONFIG['forecast_days']
INVESTMENT_AMOUNT = CONFIG['investment_amount']

📊 Analysis Configuration:
symbols             : ['FPT', 'VNM', 'HPG', 'MWG', 'VCB']
start_date          : 2022-01-01
end_date            : 2025-12-03
rf_rate             : 0.03
lookback_years      : 1
forecast_days       : 30
investment_amount   : 10000000


## 3. Tải Dữ liệu

In [4]:
print("🔄 Loading Vietnamese stock data...")
print(f"📅 Period: {START_DATE} to {END_DATE}")
print(f"📊 Symbols: {', '.join(SYMBOLS)}")

try:
    # Load stock prices
    prices = get_prices_for_list(SYMBOLS, START_DATE, END_DATE)
    returns = compute_returns(prices)
    
    # Load market data
    mkt_close, mkt_returns = get_market_index_returns(START_DATE, END_DATE)
    
    print(f"✅ Successfully loaded data for {len(prices.columns)} stocks")
    print(f"📈 Price data shape: {prices.shape}")
    print(f"📊 Returns data shape: {returns.shape}")
    print(f"📅 Date range: {prices.index.min().date()} to {prices.index.max().date()}")
    
    # Display first few rows
    print("\n📋 First 5 rows of price data:")
    display(prices.head())
    
except Exception as e:
    print(f"❌ Error loading data: {e}")
    print("🔧 Please check your internet connection and vnstock installation")

2025-12-03 02:19:27.170 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-12-03 02:19:27.172 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


🔄 Loading Vietnamese stock data...
📅 Period: 2022-01-01 to 2025-12-03
📊 Symbols: FPT, VNM, HPG, MWG, VCB


2025-12-03 02:19:28.350 
  command:

    streamlit run C:\Users\hoang\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-12-03 02:19:28.350 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-03 02:19:28.350 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-03 02:19:28.350 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-03 02:19:28.357 No runtime found, using MemoryCacheStorageManager
2025-12-03 02:19:28.870 Thread 'Thread-13': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-03 02:19:28.874 Thread 'Thread-13': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-03 02:19:28.874 Thread 'Thread-13': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-03 02:19:39.82

✅ Successfully loaded data for 5 stocks
📈 Price data shape: (1025, 5)
📊 Returns data shape: (1024, 5)
📅 Date range: 2021-10-26 to 2025-12-02

📋 First 5 rows of price data:


,FPT,VNM,HPG,MWG,VCB
time,,,,,
2021-10-26,49.18,70.20,32.28,61.60,40.92
2021-10-27,49.84,70.82,33.20,63.52,41.83
2021-10-28,49.74,70.98,33.25,63.52,42.62
2021-10-29,49.18,71.06,32.74,63.23,42.66
2021-11-01,48.63,70.51,31.93,62.56,42.23


## 4. Biểu đồ Giá Cổ phiếu

In [5]:
# Interactive price chart
fig_prices = go.Figure()

for symbol in SYMBOLS:
    if symbol in prices.columns:
        fig_prices.add_trace(go.Scatter(
            x=prices.index,
            y=prices[symbol],
            mode='lines',
            name=symbol,
            line=dict(width=2),
            hovertemplate=f'<b>{symbol}</b><br>Ngày: %{{x}}<br>Giá: %{{y:,.0f}} VND<extra></extra>'
        ))

fig_prices.update_layout(
    title='📈 Diễn biến Giá Cổ phiếu Việt Nam',
    xaxis_title='Thời gian',
    yaxis_title='Giá (VND)',
    hovermode='x unified',
    height=600,
    showlegend=True
)

fig_prices.show()

## 5. Thống kê Cơ bản

In [ ]:
# Calculate comprehensive statistics
print("📊 Calculating comprehensive stock statistics...")

try:
    summary_stats = summarize_all_stocks(returns, RF_RATE)
    
    print("\n📋 Summary Statistics:")
    print("=" * 80)
    display(summary_stats.round(4))
    
    # Key insights
    if not summary_stats.empty:
        best_return = summary_stats['Lợi nhuận TB (% năm)'].idxmax()
        lowest_risk = summary_stats['Độ lệch chuẩn (% năm)'].idxmin()
        best_sharpe = summary_stats['Sharpe Ratio'].idxmax()
        
        print(f"\n🎯 Key Performance Insights:")
        print(f"📈 Highest Return: {best_return} ({summary_stats.loc[best_return, 'Lợi nhuận TB (% năm)']:.2f}%)")
        print(f"🛡️ Lowest Risk: {lowest_risk} ({summary_stats.loc[lowest_risk, 'Độ lệch chuẩn (% năm)']:.2f}%)")
        print(f"⚡ Best Sharpe Ratio: {best_sharpe} ({summary_stats.loc[best_sharpe, 'Sharpe Ratio']:.3f})")
    
except Exception as e:
    print(f"❌ Error calculating statistics: {e}")

## 6. Ma trận Tương quan

In [ ]:
# Correlation analysis
correlation_matrix = returns.corr()

# Interactive correlation heatmap
fig_corr = px.imshow(
    correlation_matrix.values,
    x=correlation_matrix.columns,
    y=correlation_matrix.index,
    color_continuous_scale='RdBu_r',
    color_continuous_midpoint=0,
    title='📊 Ma trận Tương quan giữa các Cổ phiếu',
    labels=dict(color="Correlation")
)

# Add text annotations
for i in range(len(correlation_matrix.index)):
    for j in range(len(correlation_matrix.columns)):
        fig_corr.add_annotation(
            x=j, y=i,
            text=str(round(correlation_matrix.iloc[i, j], 3)),
            showarrow=False,
            font=dict(color="white" if abs(correlation_matrix.iloc[i, j]) > 0.5 else "black")
        )

fig_corr.update_layout(height=500)
fig_corr.show()

# Correlation insights
avg_corr = correlation_matrix.values[np.triu_indices_from(correlation_matrix.values, k=1)].mean()
max_corr = correlation_matrix.values[np.triu_indices_from(correlation_matrix.values, k=1)].max()
min_corr = correlation_matrix.values[np.triu_indices_from(correlation_matrix.values, k=1)].min()

print(f"\n📊 Correlation Analysis:")
print(f"Average Correlation: {avg_corr:.3f}")
print(f"Highest Correlation: {max_corr:.3f}")
print(f"Lowest Correlation: {min_corr:.3f}")

## 7. Phân tích Rủi ro - Lợi nhuận

In [ ]:
# Risk-Return scatter plot
risk_return_data = []

for stock in SYMBOLS:
    if stock in returns.columns:
        stock_returns = returns[stock].dropna()
        if len(stock_returns) > 20:
            annual_return = stock_returns.mean() * 252 * 100  # %
            annual_risk = stock_returns.std() * np.sqrt(252) * 100  # %
            sharpe = (annual_return - RF_RATE * 100) / annual_risk if annual_risk > 0 else 0
            
            risk_return_data.append({
                'Stock': stock,
                'Return (%)': annual_return,
                'Risk (%)': annual_risk,
                'Sharpe Ratio': sharpe
            })

risk_return_df = pd.DataFrame(risk_return_data)

if not risk_return_df.empty:
    # Create interactive scatter plot
    fig_risk_return = go.Figure()
    
    fig_risk_return.add_trace(go.Scatter(
        x=risk_return_df['Risk (%)'],
        y=risk_return_df['Return (%)'],
        mode='markers+text',
        text=risk_return_df['Stock'],
        textposition="top center",
        marker=dict(
            size=15,
            color=risk_return_df['Sharpe Ratio'],
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title="Sharpe Ratio")
        ),
        hovertemplate='<b>%{text}</b><br>Risk: %{x:.2f}%<br>Return: %{y:.2f}%<br>Sharpe: %{marker.color:.3f}<extra></extra>'
    ))
    
    fig_risk_return.update_layout(
        title='📊 Risk-Return Analysis',
        xaxis_title='Risk (Annual Volatility %)',
        yaxis_title='Expected Return (% per year)',
        height=500,
        showlegend=False
    )
    
    fig_risk_return.show()
    
    # Display data table
    print("\n📋 Risk-Return Summary:")
    display(risk_return_df.round(3))

## 8. Tối ưu hóa Danh mục - Efficient Frontier

In [ ]:
# Portfolio optimization with Efficient Frontier
print("🔄 Running Portfolio Optimization...")

try:
    # Use historical returns as expected returns
    expected_returns = []
    for stock in SYMBOLS:
        if stock in risk_return_df['Stock'].values:
            expected_ret = risk_return_df[risk_return_df['Stock'] == stock]['Return (%)'].iloc[0] / 100
            expected_returns.append(expected_ret)
        else:
            expected_returns.append(0.1)  # Default 10%
    
    print(f"📊 Expected Returns: {[f'{r*100:.1f}%' for r in expected_returns]}")
    
    # Create efficient frontier analyzer
    ef_analyzer = EfficientFrontierAnalyzer(prices, expected_returns)
    
    # Find optimal portfolios
    min_var_portfolio = ef_analyzer.find_minimum_variance_portfolio()
    max_sharpe_portfolio = ef_analyzer.find_max_sharpe_portfolio()
    
    # Build efficient frontier
    ef_analyzer.build_efficient_frontier()
    
    print(f"\n🎯 Optimal Portfolio Results:")
    print(f"Min Variance Portfolio:")
    print(f"  Return: {min_var_portfolio['return']*100:.2f}%/year")
    print(f"  Risk: {min_var_portfolio['volatility']*100:.2f}%/year")
    print(f"  Sharpe: {min_var_portfolio['sharpe']:.3f}")
    
    print(f"\nMax Sharpe Portfolio:")
    print(f"  Return: {max_sharpe_portfolio['return']*100:.2f}%/year")
    print(f"  Risk: {max_sharpe_portfolio['volatility']*100:.2f}%/year")
    print(f"  Sharpe: {max_sharpe_portfolio['sharpe']:.3f}")
    
except Exception as e:
    print(f"❌ Portfolio optimization error: {e}")

In [ ]:
# Plot efficient frontier
try:
    fig_ef = ef_analyzer.plot_efficient_frontier()
    fig_ef.update_layout(
        title="📈 Efficient Frontier - Portfolio Optimization",
        height=600
    )
    fig_ef.show()
    
except Exception as e:
    print(f"❌ Error plotting efficient frontier: {e}")

## 9. Khuyến nghị Đầu tư

In [ ]:
# Investment recommendations
try:
    print(f"💰 Investment Recommendations for {INVESTMENT_AMOUNT:,} VND")
    print("=" * 60)
    
    # Max Sharpe recommendation
    recommendation_max_sharpe, selected_portfolio, portfolio_name = ef_analyzer.get_investment_recommendation(
        INVESTMENT_AMOUNT, 'max_sharpe'
    )
    
    print(f"\n🎯 {portfolio_name}:")
    display(recommendation_max_sharpe)
    
    # Min Variance recommendation  
    recommendation_min_var, _, portfolio_name_min = ef_analyzer.get_investment_recommendation(
        INVESTMENT_AMOUNT, 'min_var'
    )
    
    print(f"\n🛡️ {portfolio_name_min}:")
    display(recommendation_min_var)
    
except Exception as e:
    print(f"❌ Error generating recommendations: {e}")

## 10. Phân tích CAPM

In [ ]:
# CAPM Analysis
print("⚖️ Running CAPM Analysis...")

try:
    capm_analyzer = CAPMAnalyzer(returns, mkt_returns, RF_RATE)
    
    # Calculate CAPM metrics for all stocks
    capm_results = []
    E_R_actual = (summary_stats["Lợi nhuận TB (% năm)"] / 100).to_dict() if not summary_stats.empty else {}
    mean_daily_mkt = mkt_returns.mean()
    E_Rm_annual = mean_daily_mkt * 252
    
    for stock in SYMBOLS:
        if stock in returns.columns:
            beta_stats = capm_analyzer.calculate_beta(stock)
            
            if 'error' not in beta_stats:
                expected_return_capm = RF_RATE + beta_stats['beta'] * (E_Rm_annual - RF_RATE)
                actual_return = E_R_actual.get(stock, 0)
                
                capm_results.append({
                    'Stock': stock,
                    'Beta': beta_stats['beta'],
                    'Actual Return (%)': actual_return * 100,
                    'CAPM Expected (%)': expected_return_capm * 100,
                    'Alpha (%)': (actual_return - expected_return_capm) * 100,
                    'R²': beta_stats['r_squared']
                })
    
    if capm_results:
        capm_df = pd.DataFrame(camp_results)
        print("\n📊 CAPM Analysis Results:")
        display(capm_df.round(4))
        
        # CAPM insights
        print("\n💡 CAPM Insights:")
        for _, row in capm_df.iterrows():
            beta_interpretation = "High Risk" if row['Beta'] > 1 else "Low Risk" if row['Beta'] < 0.7 else "Market Risk"
            alpha_interpretation = "Outperform" if row['Alpha (%)'] > 0 else "Underperform"
            print(f"{row['Stock']}: {beta_interpretation} (β={row['Beta']:.3f}), {alpha_interpretation} (α={row['Alpha (%)']:+.2f}%)")
    
except Exception as e:
    print(f"❌ CAPM analysis error: {e}")

In [ ]:
# Plot Security Market Line
try:
    fig_sml = capm_analyzer.plot_security_market_line()
    fig_sml.update_layout(
        title="⚖️ Security Market Line (SML) - CAPM Analysis",
        height=600
    )
    fig_sml.show()
    
except Exception as e:
    print(f"❌ Error plotting SML: {e}")

## 11. 🔴 Dự báo LSTM - FIXED RED LINE

In [6]:
# LSTM Forecasting
print("🔮 Running LSTM Forecasting Analysis...")

# Select stock for forecasting
forecast_stock = SYMBOLS[0]  # Use first stock, you can change this
lookback_days = int(LOOKBACK_YEARS * 252)  # Convert years to trading days

print(f"📊 LSTM Configuration:")
print(f"  Stock: {forecast_stock}")
print(f"  Lookback: {LOOKBACK_YEARS} years ({lookback_days} days)")
print(f"  Forecast: {FORECAST_DAYS} days")
print("=" * 50)

# Initialize variables
lstm_results = None
ma_results = None
stock_prices = None

try:
    if forecast_stock in prices.columns:
        stock_prices = prices[forecast_stock].dropna()
        
        if len(stock_prices) >= lookback_days + 50:
            # Run LSTM analysis
            print("🤖 Training LSTM Neural Network...")
            lstm_results = run_lstm_analysis(stock_prices, forecast_stock, lookback_days, FORECAST_DAYS)
            
            if lstm_results and lstm_results.get('success', False):
                # Extract results
                metrics = lstm_results['training']['metrics']
                forecast_results = lstm_results['forecast']
                current_price = stock_prices.iloc[-1]
                predicted_price = forecast_results['predictions'][-1]
                pred_change = (predicted_price / current_price - 1) * 100
                
                print(f"\n✅ LSTM Training Successful!")
                print(f"📊 Model Performance:")
                print(f"  RMSE: {metrics['test_rmse']:.2f}")
                print(f"  MAE: {metrics['test_mae']:.2f}")
                print(f"  Epochs: {metrics['epochs_trained']}")
                
                print(f"\n💰 Price Forecast:")
                print(f"  Current: {current_price:,.0f} VND")
                print(f"  Predicted ({FORECAST_DAYS}d): {predicted_price:,.0f} VND")
                print(f"  Expected change: {pred_change:+.2f}%")
                
                # Trend interpretation
                if pred_change > 5:
                    trend = "🚀 Strong Upward Trend"
                elif pred_change > 0:
                    trend = "📈 Moderate Upward Trend" 
                elif pred_change > -5:
                    trend = "📉 Moderate Downward Trend"
                else:
                    trend = "📉 Strong Downward Trend"
                
                print(f"  Trend: {trend}")
                
            else:
                error_msg = lstm_results.get('error', 'Unknown error') if lstm_results else 'No results returned'
                print(f"❌ LSTM failed: {error_msg}")
                print("🔄 Falling back to Moving Average forecast...")
                
                # Fallback to MA
                ma_results = simple_moving_average_forecast(stock_prices, FORECAST_DAYS)
                if ma_results and ma_results.get('success', False):
                    current_price = stock_prices.iloc[-1]
                    predicted_price = ma_results['predictions'][-1]
                    pred_change = (predicted_price / current_price - 1) * 100
                    
                    print(f"📊 Moving Average Forecast:")
                    print(f"  Current: {current_price:,.0f} VND")
                    print(f"  Predicted: {predicted_price:,.0f} VND")
                    print(f"  Change: {pred_change:+.2f}%")
        else:
            print(f"❌ Insufficient data for {forecast_stock}")
            print(f"   Required: {lookback_days + 50} days")
            print(f"   Available: {len(stock_prices)} days")
    else:
        print(f"❌ Stock {forecast_stock} not found in prices data")
    
except Exception as e:
    print(f"❌ LSTM forecasting error: {e}")
    import traceback
    traceback.print_exc()

🔮 Running LSTM Forecasting Analysis...
📊 LSTM Configuration:
  Stock: FPT
  Lookback: 1 years (252 days)
  Forecast: 30 days
🤖 Training LSTM Neural Network...

✅ LSTM Training Successful!
📊 Model Performance:
  RMSE: 3.87
  MAE: 3.01
  Epochs: 30

💰 Price Forecast:
  Current: 97 VND
  Predicted (30d): 99 VND
  Expected change: +2.79%
  Trend: 📈 Moderate Upward Trend


In [7]:
# 🔴 LSTM FORECAST VISUALIZATION - FIXED RED LINE
print("\n🎨 Creating forecast visualization...")

if lstm_results and lstm_results.get('success', False) and stock_prices is not None:
    try:
        print("📊 Creating LSTM forecast chart with RED LINE...")
        
        fig_lstm = go.Figure()
        
        # Historical data (recent period for better visualization)
        display_days = min(lookback_days // 2, len(stock_prices), 100)
        recent_prices = stock_prices.tail(display_days)
        
        # Add historical prices (BLUE)
        fig_lstm.add_trace(go.Scatter(
            x=recent_prices.index,
            y=recent_prices.values,
            mode='lines',
            name=f'{forecast_stock} Historical',
            line=dict(color='blue', width=2),
            hovertemplate='<b>Historical</b><br>Date: %{x}<br>Price: %{y:,.0f} VND<extra></extra>'
        ))
        
        # Extract LSTM forecast data
        forecast_results = lstm_results['forecast']
        
        if 'predictions' in forecast_results and len(forecast_results['predictions']) > 0:
            forecast_predictions = np.array(forecast_results['predictions']).flatten()
            
            # Get or create forecast dates
            if 'dates' in forecast_results and len(forecast_results['dates']) == len(forecast_predictions):
                forecast_dates = forecast_results['dates']
            else:
                # Create forecast dates manually
                last_date = recent_prices.index[-1]
                
                # Robust date handling
                try:
                    if isinstance(last_date, (int, np.integer)):
                        last_date = pd.Timestamp.today()
                    elif isinstance(last_date, str):
                        last_date = pd.to_datetime(last_date)
                    elif hasattr(last_date, 'date') or isinstance(last_date, pd.Timestamp):
                        last_date = pd.to_datetime(last_date)
                    else:
                        last_date = pd.Timestamp.today()
                except:
                    last_date = pd.Timestamp.today()
                
                forecast_dates = pd.date_range(
                    start=last_date + pd.Timedelta(days=1),
                    periods=len(forecast_predictions),
                    freq='D'
                )
            
            # Connection line from last historical to first forecast (ORANGE)
            fig_lstm.add_trace(go.Scatter(
                x=[recent_prices.index[-1], forecast_dates[0]],
                y=[recent_prices.iloc[-1], forecast_predictions[0]],
                mode='lines',
                line=dict(color='orange', width=2, dash='dot'),
                name='Connection',
                showlegend=False,
                hoverinfo='skip'
            ))
            
            # 🔴 LSTM FORECAST LINE - ENHANCED FOR VISIBILITY
            fig_lstm.add_trace(go.Scatter(
                x=forecast_dates,
                y=forecast_predictions,
                mode='lines+markers',
                name='🔴 LSTM Forecast',
                line=dict(
                    color='red', 
                    width=4,  # Thicker line
                    dash=None  # Solid line
                ),
                marker=dict(
                    size=8, 
                    color='red',
                    symbol='circle',
                    line=dict(width=2, color='darkred')
                ),
                visible=True,  # Force visible
                opacity=1.0,   # Full opacity
                hovertemplate='<b>🔴 LSTM Forecast</b><br>Date: %{x}<br>Price: %{y:,.0f} VND<extra></extra>'
            ))
            
            # Confidence interval (light red shading)
            ci = forecast_results.get('confidence_interval', {})
            if 'upper' in ci and 'lower' in ci and len(ci['upper']) == len(forecast_dates):
                # Upper bound (invisible line)
                fig_lstm.add_trace(go.Scatter(
                    x=forecast_dates,
                    y=ci['upper'],
                    fill=None,
                    mode='lines',
                    line_color='rgba(0,0,0,0)',
                    showlegend=False,
                    hoverinfo='skip'
                ))
                
                # Lower bound with fill
                fig_lstm.add_trace(go.Scatter(
                    x=forecast_dates,
                    y=ci['lower'],
                    fill='tonexty',
                    mode='lines',
                    line_color='rgba(0,0,0,0)',
                    fillcolor='rgba(255,0,0,0.15)',  # Light red fill
                    name='Confidence Interval',
                    hovertemplate='Confidence Interval<extra></extra>'
                ))
            
            # Chart styling
            fig_lstm.update_layout(
                title={
                    'text': f'🔮 LSTM Forecast for {forecast_stock} - {FORECAST_DAYS} days prediction',
                    'x': 0.5,
                    'font': {'size': 16}
                },
                xaxis_title='Date',
                yaxis_title='Price (VND)',
                height=650,
                hovermode='x unified',
                showlegend=True,
                legend=dict(
                    x=0.01, y=0.99,
                    bgcolor='rgba(255,255,255,0.8)',
                    bordercolor='gray',
                    borderwidth=1
                ),
                template='plotly_white'
            )
            
            # Add annotation for forecast summary
            current_price = recent_prices.iloc[-1]
            final_forecast = forecast_predictions[-1]
            change_pct = (final_forecast / current_price - 1) * 100
            
            fig_lstm.add_annotation(
                x=0.02, y=0.98,
                xref="paper", yref="paper",
                text=f"<b>Forecast Summary</b><br>" +
                     f"Current: {current_price:,.0f} VND<br>" +
                     f"30-day: {final_forecast:,.0f} VND<br>" +
                     f"Change: {change_pct:+.1f}%",
                showarrow=False,
                font=dict(size=11, color="black"),
                bgcolor="rgba(255,255,255,0.8)",
                bordercolor="gray",
                borderwidth=1
            )
            
            fig_lstm.show()
            
            print(f"\n✅ LSTM chart displayed successfully!")
            print(f"🔴 Red forecast line should be clearly visible")
            print(f"📊 Chart includes:")
            print(f"   - Historical: {len(recent_prices)} points (blue)")
            print(f"   - Forecast: {len(forecast_predictions)} points (red)")
            print(f"   - Date range: {forecast_dates[0].strftime('%Y-%m-%d')} to {forecast_dates[-1].strftime('%Y-%m-%d')}")
            
        else:
            print("❌ No forecast predictions found")
            
    except Exception as e:
        print(f"❌ Error creating LSTM chart: {e}")
        import traceback
        traceback.print_exc()

elif ma_results and ma_results.get('success', False) and stock_prices is not None:
    try:
        print("📊 Creating Moving Average forecast chart...")
        
        fig_ma = go.Figure()
        
        # Historical data
        recent_prices = stock_prices.tail(100)
        fig_ma.add_trace(go.Scatter(
            x=recent_prices.index,
            y=recent_prices.values,
            mode='lines',
            name=f'{forecast_stock} Historical',
            line=dict(color='blue', width=2)
        ))
        
        # MA forecast
        if 'dates' in ma_results and 'predictions' in ma_results:
            fig_ma.add_trace(go.Scatter(
                x=ma_results['dates'],
                y=ma_results['predictions'],
                mode='lines+markers',
                name='🔴 MA Forecast',
                line=dict(color='red', width=4),
                marker=dict(size=6, color='red')
            ))
        
        fig_ma.update_layout(
            title=f'📊 Moving Average Forecast for {forecast_stock}',
            xaxis_title='Date',
            yaxis_title='Price (VND)',
            height=600
        )
        
        fig_ma.show()
        print("✅ MA forecast chart displayed")
        
    except Exception as e:
        print(f"❌ Error creating MA chart: {e}")

else:
    print("❌ No forecast results available for visualization")
    if lstm_results:
        print(f"LSTM success: {lstm_results.get('success', False)}")
        if not lstm_results.get('success', False):
            print(f"LSTM error: {lstm_results.get('error', 'No error info')}")
    else:
        print("No LSTM results generated")


🎨 Creating forecast visualization...
📊 Creating LSTM forecast chart with RED LINE...



✅ LSTM chart displayed successfully!
🔴 Red forecast line should be clearly visible
📊 Chart includes:
   - Historical: 100 points (blue)
   - Forecast: 30 points (red)
   - Date range: 2025-12-03 to 2026-01-01


## 12. Tổng kết và Lưu kết quả

In [8]:
print("🎯 INVESTMENT ANALYSIS SUMMARY")
print("=" * 60)

# Stock performance summary
if 'summary_stats' in locals() and not summary_stats.empty:
    print("\n📈 STOCK PERFORMANCE:")
    for stock in SYMBOLS:
        if stock in summary_stats.index:
            row = summary_stats.loc[stock]
            print(f"   {stock}: {row['Lợi nhuận TB (% năm)']:+.2f}% return, {row['Độ lệch chuẩn (% năm)']:.2f}% risk, Sharpe {row['Sharpe Ratio']:.3f}")

# Portfolio recommendations summary
if 'min_var_portfolio' in locals() and 'max_sharpe_portfolio' in locals():
    print("\n💼 PORTFOLIO RECOMMENDATIONS:")
    print(f"   Conservative (Min Risk): {min_var_portfolio['return']*100:.2f}% return, {min_var_portfolio['volatility']*100:.2f}% risk")
    print(f"   Aggressive (Max Sharpe): {max_sharpe_portfolio['return']*100:.2f}% return, {max_sharpe_portfolio['volatility']*100:.2f}% risk")

# CAPM insights summary
if 'capm_df' in locals() and not capm_df.empty:
    print("\n⚖️ RISK ANALYSIS (CAPM):")
    for _, row in capm_df.iterrows():
        risk_level = "High" if row['Beta'] > 1.2 else "Low" if row['Beta'] < 0.8 else "Moderate"
        performance = "outperform" if row['Alpha (%)'] > 1 else "underperform" if row['Alpha (%)'] < -1 else "neutral"
        print(f"   {row['Stock']}: {risk_level} risk (β={row['Beta']:.2f}), tends to {performance} market")

# LSTM forecast summary
if lstm_results and lstm_results.get('success', False):
    forecast_results = lstm_results['forecast']
    if 'predictions' in forecast_results and len(forecast_results['predictions']) > 0:
        current_price = stock_prices.iloc[-1]
        predicted_price = forecast_results['predictions'][-1]
        pred_change = (predicted_price / current_price - 1) * 100
        
        print(f"\n🔮 PRICE FORECAST ({forecast_stock}):")
        print(f"   Current: {current_price:,.0f} VND")
        print(f"   {FORECAST_DAYS}-day forecast: {predicted_price:,.0f} VND ({pred_change:+.1f}%)")
        
        if pred_change > 2:
            recommendation = "🟢 BUY signal - Upward trend predicted"
        elif pred_change < -2:
            recommendation = "🔴 SELL signal - Downward trend predicted"
        else:
            recommendation = "🟡 HOLD signal - Sideways movement expected"
        
        print(f"   Recommendation: {recommendation}")

print("\n⚠️  DISCLAIMER:")
print("   This analysis is based on historical data and mathematical models.")
print("   Past performance does not guarantee future results.")
print("   Always conduct additional research before making investment decisions.")
print("   Consider consulting with a financial advisor.")

🎯 INVESTMENT ANALYSIS SUMMARY

🔮 PRICE FORECAST (FPT):
   Current: 97 VND
   30-day forecast: 99 VND (+2.8%)
   Recommendation: 🟢 BUY signal - Upward trend predicted

⚠️  DISCLAIMER:
   This analysis is based on historical data and mathematical models.
   Past performance does not guarantee future results.
   Always conduct additional research before making investment decisions.
   Consider consulting with a financial advisor.


In [9]:
# Save results to Excel
try:
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_file = f'stock_analysis_results_{timestamp}.xlsx'
    
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        # Save all dataframes
        if 'summary_stats' in locals() and not summary_stats.empty:
            summary_stats.to_excel(writer, sheet_name='Summary_Statistics')
        
        if 'risk_return_df' in locals() and not risk_return_df.empty:
            risk_return_df.to_excel(writer, sheet_name='Risk_Return_Analysis', index=False)
        
        if 'correlation_matrix' in locals():
            correlation_matrix.to_excel(writer, sheet_name='Correlation_Matrix')
        
        if 'capm_df' in locals() and not camp_df.empty:
            camp_df.to_excel(writer, sheet_name='CAPM_Analysis', index=False)
        
        if 'recommendation_max_sharpe' in locals():
            recommendation_max_sharpe.to_excel(writer, sheet_name='Max_Sharpe_Portfolio', index=False)
            
        if 'recommendation_min_var' in locals():
            recommendation_min_var.to_excel(writer, sheet_name='Min_Variance_Portfolio', index=False)
        
        if 'prices' in locals():
            prices.to_excel(writer, sheet_name='Price_Data')
            
        if 'returns' in locals():
            returns.to_excel(writer, sheet_name='Returns_Data')
    
    print(f"\n💾 Results saved to: {output_file}")
    print(f"📁 File size: {os.path.getsize(output_file) / 1024:.2f} KB")
    
    # Save configuration
    import json
    config_file = f'analysis_config_{timestamp}.json'
    config_with_timestamp = CONFIG.copy()
    config_with_timestamp['analysis_timestamp'] = datetime.now().isoformat()
    config_with_timestamp['random_seed'] = 42
    
    with open(config_file, 'w', encoding='utf-8') as f:
        json.dump(config_with_timestamp, f, indent=2, ensure_ascii=False)
    
    print(f"⚙️  Configuration saved to: {config_file}")
    
except Exception as e:
    print(f"❌ Error saving results: {e}")


💾 Results saved to: stock_analysis_results_20251203_022505.xlsx
📁 File size: 110.31 KB
⚙️  Configuration saved to: analysis_config_20251203_022505.json
